In [1]:
import torch
from model.TGS_UNet_ResNet34 import *
from utils.RLE import Encode
import cv2
import torch.utils.data as data
import data.TestDatasets as TestDatasets
import os
import pandas as pd
import numpy as np
import torchvision.models as model

In [2]:
resnet = model.resnet34(pretrained=True)
unet_100 = UNet(resnet)

In [3]:
unet_100.load_state_dict(torch.load('model_unet_params.pkl'))

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
test_image_root = './input/test/'
test_image_name = os.listdir(test_image_root)
test_dataset = TestDatasets.my_TestDatasets(test_image_root,test_image_name)
test_loader = data.DataLoader(test_dataset,batch_size=16)
unet_100 = unet_100.to(device)

In [6]:
submission = pd.DataFrame(columns=('id','rle_mask'))

In [7]:
def rleEncode(img):
    img = cv2.resize(img,(101,101),interpolation=cv2.INTER_CUBIC)
    img = np.int32(img>0)
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [8]:
unet_100.eval()
with torch.no_grad():
    for i,(img,name) in enumerate(test_loader):
        img = img.to(device)
        outputs = unet_100(img)
        del img
        outputs = (outputs>0).float()
        outputs=outputs.cpu()
        outputs = outputs.numpy()
        for j,(out,img_name) in enumerate(zip(outputs,name)):
            rle = rleEncode(out.transpose())
            img_name = img_name.split('.')[0]
            submission.loc[i*16+j]={'id':img_name,'rle_mask':rle}

In [9]:
submission.head()

,id,rle_mask
0,0005bb9630,2 15 19 1 21 97 119 1 121 83 205 8 214 2 218 2...
1,000a68e46c,
2,000c8dfb2a,
3,000d0a5f6c,
4,001ef8fc87,1 7361 7363 2 7366 4 7374 82 7463 1 7466 1 747...


In [10]:
submission.to_csv('submission.csv',index=False)

In [11]:
len(submission)

18000